# Pré-processamento de dados de dados

Esse código realiza o pré-processamento dos dados de interação do usuário, gerando uma matriz de interação e transformando informações históricas em um formato estruturado.

Principais funções

✅ calc_recency_score(issued): Calcula a pontuação de recência de notícias, dando mais peso às mais recentes. \
✅ parse_date_timestamp(date): Converte timestamps para milissegundos, garantindo consistência no formato. \
✅ build_interaction_matrix(df, interaction_columns, chunk_size=10000): Constrói uma matriz esparsa de interação usuário-notícia em chunks, útil para grandes volumes de dados. \
✅ transform_user_to_info_user_item(user_info): Transforma os dados brutos do usuário em um formato mais estruturado e pronto para análise.

In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

interaction_columns = [
    'pageVisitsCountHistory',
    'scrollPercentageHistory',
]

def calc_recency_score(issued: pd.Series):
    """
    Calcula o score de recência para uma série de timestamps.

    O score é inversamente proporcional ao tempo decorrido,
    dando mais prioridade a notícias mais recentes.

    :param issued: Série com os timestamps das notícias.
    :return: Série com os scores de recência.
    """
    # Converter os timestamps para milissegundos

    now = parse_date_timestamp(datetime.timestamp(datetime.now()))

    # Calcular o score de recência, evitando divisão por zero
    return 1 / np.maximum(now - parse_date_timestamp(issued), 1e-6)


def parse_date_timestamp(date):
    # Converte para datetime se ainda for string
    if isinstance(date, str):
        date = pd.to_datetime(date, errors='coerce')
    elif isinstance(date, float):
        return (date * 1000)

    return int(datetime.timestamp(date) * 1000)


def build_interaction_matrix(df, interaction_columns, chunk_size=10000):
    """
    Constrói uma matriz de interação a partir de um DataFrame específico em chunks.
    Garante que todas as matrizes parciais tenham as mesmas colunas automaticamente.
    Retorna uma matriz esparsa empilhada verticalmente.
    """
    interaction_matrix = []

    # Extrai todas as colunas únicas automaticamente do DataFrame
    all_columns = df['history'].unique()
    print(f"Colunas únicas encontradas: {len(all_columns)}")

    for start in tqdm(range(0, len(df), chunk_size), desc='Building interaction matrix'):
        chunk = df.iloc[start:start + chunk_size]

        # Gera a matriz de interação parcial
        partial_matrix = chunk.pivot_table(
            index='userId',
            columns='history',
            values=interaction_columns,
            aggfunc='mean',
            fill_value=0
        )

        # Reindexa para garantir a consistência das colunas
        partial_matrix = partial_matrix.reindex(columns=all_columns, fill_value=0)

        # Converte para matriz esparsa e acumula
        interaction_matrix.append(csr_matrix(partial_matrix))

    # Retorna a matriz esparsa empilhada verticalmente
    return vstack(interaction_matrix)


def transform_user_to_info_user_item(user_info):
    user_item = user_info[['userId', 'history', 'timestampHistory', 'numberOfClicksHistory', 'timeOnPageHistory', 'scrollPercentageHistory', 'pageVisitsCountHistory', 'timestampHistory_new']]

    user_item = user_item.set_index('userId').apply(lambda row: row.str.split(','), axis=1)
    user_item = user_item.apply(pd.Series.explode).reset_index()
    user_item['numberOfClicksHistory'] = user_item['numberOfClicksHistory'].astype(int)
    user_item['timeOnPageHistory'] = user_item['timeOnPageHistory'].astype(int)
    user_item['scrollPercentageHistory'] = user_item['scrollPercentageHistory'].astype(float)
    user_item['timestampHistory'] = pd.to_numeric(user_item['timestampHistory'], errors='coerce').fillna(0).astype(int)
    user_item['pageVisitsCountHistory'] = pd.to_numeric(user_item['pageVisitsCountHistory'], errors='coerce').fillna(0).astype(int)
    user_item['history'] = user_item['history'].str.strip()
    user_item['userId'] = user_item['userId'].str.strip()
    return user_item


Esse código processa múltiplos arquivos CSV contendo dados de interação do usuário, transforma-os e constrói matrizes esparsas para análise.

O que ele faz? \
Percorre todos os arquivos CSV na pasta ../data/raw/files/treino/. \
Lê e transforma os dados de cada arquivo com transform_user_to_info_user_item(df). \
Constrói uma matriz esparsa de interação com build_interaction_matrix(df, interaction_columns). \
Acumula os resultados em listas (interaction_matrices e user_infos).

In [ ]:
import pandas as pd
import glob
from scipy.sparse import csr_matrix

# Lista para acumular as matrizes parciais
interaction_matrices = []

user_infos = []

for fpath in glob.glob('../data/raw/files/treino/*.csv'):
    print(f"Processando: {fpath}")
    df = pd.read_csv(fpath)
    df = transform_user_to_info_user_item(df)

    interaction_matrices.append(build_interaction_matrix(df, interaction_columns))
    user_infos.append(df)

O que ele faz? \
Concatena todos os DataFrames de usuários (user_info). \
Garante um conjunto fixo de colunas para padronizar os dados (all_columns). \
Empilha verticalmente (vstack) as matrizes de interação geradas nos arquivos CSV. \
Normaliza os dados com StandardScaler para manter os valores na mesma escala. \
Converte novamente para uma matriz esparsa (csr_matrix), reduzindo consumo de memória. \
Cria categorias (Categorical) para usuários e histórico, permitindo indexação eficiente.

In [ ]:
from scipy.sparse import vstack
from sklearn.preprocessing import StandardScaler
import pandas as pd

user_info = pd.concat(user_infos, ignore_index=True)

# Define um conjunto fixo de colunas baseado no DataFrame completo
all_columns = user_infos['history'].unique()
all_columns = pd.Index(all_columns)

# Concatena todas as matrizes esparsas em uma matriz final
if interaction_matrices:
    print("Concatenando matrizes...")
    interaction_matrix = vstack(interaction_matrices)

    # Normaliza e converte para matriz esparsa
    print("Normalizando e convertendo para matriz esparsa...")
    interaction_matrix = StandardScaler().fit_transform(interaction_matrix.toarray())
    interaction_matrix = csr_matrix(interaction_matrix)

    # Extrai categorias de usuários e itens
    user_id_category = pd.Categorical(range(interaction_matrix.shape[0]))
    history_id_category = pd.Categorical(range(interaction_matrix.shape[1]))

    print("Matriz de interação criada com sucesso!")
else:
    print("Nenhum dado processado!")

In [ ]:
news_item = pd.concat([pd.read_csv(fpath) for fpath in glob.glob('../data/raw/itens/itens/*.csv')])
news_item.head()

## Criando score de recencia e score de populariadade por noticia

Esse código calcula dois scores para os itens de notícias:

recency_score (Score de Recência)

Converte a coluna issued para datetime.
Aplica a função calc_recency_score() para calcular a recência de cada notícia.
Normaliza os valores com MinMaxScaler() para ficarem entre 0 e 1.
popularity_score (Score de Popularidade)

Conta quantas vezes cada notícia aparece no histórico dos usuários.
Junta essa contagem (count_visits_by_news) ao news_item.
Renomeia a coluna pageVisitsCountHistory para popularity_score.
Por fim, organiza os dados e exibe as primeiras linhas (head()).

In [ ]:
scaler = MinMaxScaler()

count_visits_by_news = (user_infos
                        .groupby('history')['pageVisitsCountHistory']
                        .count())
news_item['issued'] = pd.to_datetime(news_item['issued'], errors='coerce')
news_item['recency_score'] = scaler.fit_transform(
    news_item['issued'].apply(calc_recency_score).fillna(0).values.reshape(-1, 1)
)


news_item = (news_item.set_index('page').join(count_visits_by_news, how='inner').reset_index())
news_item.rename(columns={'pageVisitsCountHistory': 'popularity_score'}, inplace=True)
news_item.rename(columns={'index': 'history'},inplace=True)
news_item.head()

In [ ]:
user_infos.query("userId == 'ec1639851d99586c7f4da928deb49187303aec6e3b8d66c0359d4920e3c105e6' and history in @filtros")

Percebemos que o usuário não possui um vínculo forte com a notícia, ou seja, ele não expressa uma opinião clara sobre ela. Isso pode gerar desafios, pois ele pode ter visualizado a notícia sem interesse ou, por outro lado, ainda não ter visto e ter interesse em acessá-la.

Diante disso, concluímos que a melhor abordagem seria utilizar... (veja o próximo notebook para descobrir!)